## 下载数据

图片按照如下目录结构组织：

![](images/2022-01-13-16-08-29.png)

方便 `ImageDataGenerator` 提取数据。


In [1]:
import urllib.request
import zipfile

url = (
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
)

file_name = "horse-or-human.zip"
training_dir = "horse-or-human/training/"
urllib.request.urlretrieve(url, file_name)

zip_ref = zipfile.ZipFile(file_name, "r")
zip_ref.extractall(training_dir)
zip_ref.close()



In [2]:
from tensorflow.keras.preprocessing import image

# 所有图片缩放到 1./255
train_datagen = image.ImageDataGenerator(rescale=1 / 255.0)
train_generator = train_datagen.flow_from_directory(
    training_dir, 
    target_size=(300, 300), 
    class_mode="binary"
)


Found 1027 images belonging to 2 classes.


和 Fashion MNIST 不同的是，这里的图片是 300x300 的彩色图片，所以可能需要更多层，通道数也从 1 到 3.

另外，只有两种类型，因此输出神经元只需要一个。

In [3]:
import tensorflow.keras as keras

model = keras.models.Sequential(
    [
        keras.layers.Conv2D(
            16, (3, 3), activation="relu", input_shape=(300, 300, 3)
        ),  # 底层特征较少
        keras.layers.MaxPooling2D(2, 2),
        keras.layers.Conv2D(32, (3, 3), activation="relu"),
        keras.layers.MaxPooling2D(2, 2),
        keras.layers.Conv2D(64, (3, 3), activation="relu"),
        keras.layers.MaxPooling2D(2, 2),
        keras.layers.Conv2D(64, (3, 3), activation="relu"),
        keras.layers.MaxPooling2D(2, 2),
        keras.layers.Conv2D(64, (3, 3), activation="relu"),
        keras.layers.MaxPooling2D(2, 2),
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [5]:
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
    metrics=["accuracy"],
)


In [6]:
history = model.fit_generator(train_generator, epochs=15)


c:\Users\dell\.conda\envs\dltf\lib\site-packages\tensorflow\python\keras\engine\training.py:1969: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
33/33 [==============================] - 24s 696ms/step - loss: 0.8539 - accuracy: 0.6018
Epoch 2/15
33/33 [==============================] - 12s 363ms/step - loss: 0.3515 - accuracy: 0.8763
Epoch 3/15
33/33 [==============================] - 12s 358ms/step - loss: 0.1751 - accuracy: 0.9357
Epoch 4/15
33/33 [==============================] - 11s 342ms/step - loss: 0.2180 - accuracy: 0.9455
Epoch 5/15
33/33 [==============================] - 11s 333ms/step - loss: 0.0990 - accuracy: 0.9649
Epoch 6/15
33/33 [==============================] - 12s 352ms/step - loss: 0.1652 - accuracy: 0.9708
Epoch 7/15
33/33 [==============================] - 12s 346ms/step - loss: 0.0305 - accuracy: 0.9932
Epoch 8/15
33/33 [==============================] - 12s 345ms/step - loss: 0.6679 - accuracy: 0.9786
Epoch 9/15
33/33 [==============================] - 12s 346ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 10/15
33/33 [==============================] - 11s 330ms/step - loss: 0.0491 - accura

In [7]:
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"

validation_file_name = "validation-horse-or-human.zip"
validation_dir = "horse-or-human/validation/"

urllib.request.urlretrieve(validation_url, validation_file_name)

zip_ref = zipfile.ZipFile(validation_file_name, "r")
zip_ref.extractall(validation_dir)
zip_ref.close()


In [8]:
validation_datagen = image.ImageDataGenerator(rescale=1 / 255.0)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, target_size=(300, 300), class_mode="binary"
)


Found 256 images belonging to 2 classes.


In [9]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator
)

Epoch 1/15
33/33 [==============================] - 14s 401ms/step - loss: 0.5383 - accuracy: 0.9864 - val_loss: 1.6624 - val_accuracy: 0.8477
Epoch 2/15
33/33 [==============================] - 13s 379ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.7824 - val_accuracy: 0.8711
Epoch 3/15
33/33 [==============================] - 13s 376ms/step - loss: 0.1322 - accuracy: 0.9873 - val_loss: 0.8549 - val_accuracy: 0.8984
Epoch 4/15
33/33 [==============================] - 12s 371ms/step - loss: 0.0210 - accuracy: 0.9951 - val_loss: 1.5994 - val_accuracy: 0.8672
Epoch 5/15
33/33 [==============================] - 13s 381ms/step - loss: 2.4180e-04 - accuracy: 1.0000 - val_loss: 1.7828 - val_accuracy: 0.8906
Epoch 6/15
33/33 [==============================] - 12s 366ms/step - loss: 0.0812 - accuracy: 0.9903 - val_loss: 2.1352 - val_accuracy: 0.8711
Epoch 7/15
33/33 [==============================] - 12s 360ms/step - loss: 9.2815e-04 - accuracy: 1.0000 - val_loss: 3.1212 - val_accuracy

In [11]:
from tensorflow.keras.preprocessing import image
import numpy as np

# 载入图片，并 reshape 为 (300,300)
img = image.load_img("images/white-horse-gb72298424_640.jpg", target_size=(300, 300))
# 转换为 2D 数组
x = image.img_to_array(img)
# 扩展维度，转换为 3D 数组
x = np.expand_dims(x, axis=0)

image_tensor = np.vstack([x])
classes = model.predict(image_tensor)
print(classes)
print(classes[0])

if classes[0] > 0.5:
    print("is a human")
else:
    print("is a horse")


[[0.]]
[0.]
is a horse


In [12]:
# 载入图片，并 reshape 为 (300,300)
img = image.load_img("images/white-horse-gb72298424_640.jpg", target_size=(300, 300))
# 转换为 2D 数组
x = image.img_to_array(img)
# 扩展维度，转换为 3D 数组
x = np.expand_dims(x, axis=0)

classes = model.predict(x)
print(classes)
print(classes[0])

if classes[0] > 0.5:
    print("is a human")
else:
    print("is a horse")


[[0.]]
[0.]
is a horse
